# train

In [1]:
import yaml
import os

with open("./config.yaml", 'rb') as f:
    config = yaml.load(f)

/home/hiroki/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """


In [2]:
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
os.makedirs(OUTPUT_ROOT, exist_ok=True)

## load library

In [3]:
# python default library
import os
import shutil
import datetime
import sys
import pickle

# general analysis tool-kit
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt

# pytorch
import torch
from torch import optim, nn
from torch.utils.tensorboard import SummaryWriter

from torchsummary import summary

# etc
import yaml
yaml.warnings({'YAMLLoadWarning': False})
import mlflow
from collections import defaultdict
from scipy.stats import zscore

# original library

import common as com
import pytorch_modeler as modeler
from pytorch_model import DAGMM as Model
from pytorch_utils import filtered_load_model
import models

import librosa
import IPython
import librosa.display

## load config and set logger

In [4]:
#with open("./config.yaml", 'rb') as f:
#    config = yaml.load(f)

log_folder = config['IO_OPTION']['OUTPUT_ROOT']+'/{0}.log'.format(datetime.date.today())
logger = com.setup_logger(log_folder, '00_train.py')

/home/hiroki/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Setting

In [5]:
# Setting seed
modeler.set_seed(42)

In [6]:
############################################################################
# Setting I/O path
############################################################################
# input dirs
INPUT_ROOT = config['IO_OPTION']['INPUT_ROOT']
dev_path = INPUT_ROOT + "/dev_data"
add_dev_path = INPUT_ROOT + "/add_dev_data"
# machine type
MACHINE_TYPE = config['IO_OPTION']['MACHINE_TYPE']
machine_types = os.listdir(dev_path)
# output dirs
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
MODEL_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/models'
TB_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/tb'
PKL_DIR = OUTPUT_ROOT + '/pkl'
#os.makedirs(OUTPUT_ROOT, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(TB_DIR, exist_ok=True)
os.makedirs(PKL_DIR, exist_ok=True)
# copy config
shutil.copy('./config.yaml', OUTPUT_ROOT)

'/media/hiroki/working/research/dcase2020/result/2D/DAGMM/strict_comp/latent16_mixture3_batch_norm/config.yaml'

## make path list and train/valid split

In [7]:
############################################################################
# make path set and train/valid split
############################################################################
'''
train_paths[machine_type]['train' or 'valid'] = path
'''
dev_train_paths = {}
add_train_paths = {}
train_paths = {}

In [8]:
for machine_type in machine_types:
    # dev train
    dev_train_paths = ["{}/{}/train/".format(dev_path, machine_type) + file for file in os.listdir("{}/{}/train".format(dev_path, machine_type))]
    dev_train_paths = sorted(dev_train_paths)
    # add_dev train
    add_train_paths = ["{}/{}/train/".format(add_dev_path, machine_type) + file for file in os.listdir("{}/{}/train".format(add_dev_path, machine_type))]
    add_train_paths = sorted(add_train_paths)
    # valid
    dev_valid_paths = ["{}/{}/test/".format(dev_path, machine_type) + file for file in os.listdir("{}/{}/test".format(dev_path, machine_type))]
    dev_valid_paths = sorted(dev_valid_paths)
    
    train_paths[machine_type] = {}
    train_paths[machine_type]['train'] = dev_train_paths + add_train_paths
    train_paths[machine_type]['valid'] = dev_valid_paths

## training

In [9]:
#############################################################################
# run
#############################################################################
def run(machine_type):
    com.tic()
    logger.info('TARGET MACHINE_TYPE: {0}'.format(machine_type))
    logger.info('MAKE DATA_LOADER')
    # dev_train_paths
    dataloaders_dict = modeler.make_dataloader(train_paths, machine_type)
    # define writer for tensorbord
    os.makedirs(TB_DIR+'/'+machine_type, exist_ok=True)         # debug
    tb_log_dir = TB_DIR + '/' + machine_type
    writer = SummaryWriter(log_dir = tb_log_dir)
    logger.info('TRAINING')
    # parameter setting
    net = Model(sample_rate=config['preprocessing']['sample_rate'],
                window_size=config['preprocessing']['window_size'],
                hop_size=config['preprocessing']['hop_size'],
                mel_bins=config['preprocessing']['mel_bins'],
                fmin=config['preprocessing']['fmin'],
                fmax=config['preprocessing']['fmax'],
                latent_size=config['fit']['latent_size'],
                mixture_size=config['fit']['mixture_size'])
    #pretrained_dict = torch.load(config['IO_OPTION']['PREMODEL_PATH'])
    #net = filtered_load_model(net, pretrained_dict)
    optimizer = optim.Adam(net.parameters(), lr=1e-3)
    num_epochs = config['fit']['num_epochs']
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e2, 
                                              max_lr=1e-3, epochs=num_epochs, steps_per_epoch=len(dataloaders_dict['train']))
    criterion = nn.MSELoss()
    try:
        history = modeler.train_net(net, dataloaders_dict, criterion, optimizer, scheduler, num_epochs, writer)
        # output
        model = history['model']
        gmm_param = history['gmm_param']
        model_out_path = MODEL_DIR+'/{}_model.pth'.format(machine_type)
        gmm_param_path = MODEL_DIR+'/{}_gmm_param.pkl'.format(machine_type)
        torch.save(model.state_dict(), model_out_path)
        logger.info('\n success:{0} \n'.format(machine_type) + \
                        'model_out_path ==> \n {0}'.format(model_out_path))
        pd.to_pickle(gmm_param, gmm_param_path)
        #  close writer for tensorbord
        writer.close()
        #modeler.mlflow_log(history, config, machine_type, model_out_path, tb_log_dir)
        com.toc()
    except:
        print("error")
        history = "error"
    return history

In [10]:
machine_types

['fan', 'pump', 'slider', 'ToyCar', 'ToyConveyor', 'valve']

In [11]:
for machine_type in machine_types:
    history = run(machine_type)
    #with open('{}/{}_history.pkl'.format(PKL_DIR, machine_type), 'wb') as file:
    #    pickle.dump(history , file)

2020-11-30 22:44:18,964 - 00_train.py - INFO - TARGET MACHINE_TYPE: fan
2020-11-30 22:44:18,965 - 00_train.py - INFO - MAKE DATA_LOADER
2020-11-30 22:44:19,788 - 00_train.py - INFO - TRAINING
  0%|          | 0/51 [00:00<?, ?it/s]

use: cuda:0


100%|██████████| 15/15 [00:01<00:00,  7.86it/s]
2020-11-30 22:44:51,719 - pytorch_modeler.py - INFO - Epoch 1/300:train_loss:48348.507996, tr_rec:48249.741394, tr_eng:792.180138, tr_covd:3909.672585, val_AUC:0.502790, val_pAUC:0.498988
100%|██████████| 15/15 [00:01<00:00,  8.02it/s]
2020-11-30 22:45:22,161 - pytorch_modeler.py - INFO - Epoch 2/300:train_loss:47972.970398, tr_rec:47906.884583, tr_eng:468.113750, tr_covd:3854.878761, val_AUC:0.487741, val_pAUC:0.504844
100%|██████████| 15/15 [00:01<00:00,  8.32it/s]
2020-11-30 22:45:52,463 - pytorch_modeler.py - INFO - Epoch 3/300:train_loss:47498.253662, tr_rec:47448.653992, tr_eng:300.516018, tr_covd:3909.549820, val_AUC:0.476325, val_pAUC:0.503465
100%|██████████| 15/15 [00:01<00:00,  8.24it/s]
2020-11-30 22:46:22,523 - pytorch_modeler.py - INFO - Epoch 4/300:train_loss:47023.222046, tr_rec:46979.572083, tr_eng:244.519816, tr_covd:3839.617104, val_AUC:0.479262, val_pAUC:0.500882
100%|██████████| 15/15 [00:01<00:00,  8.16it/s]
2020-11-

100%|██████████| 15/15 [00:02<00:00,  5.02it/s]
2020-11-30 23:07:09,779 - pytorch_modeler.py - INFO - Epoch 30/300:train_loss:2763.500822, tr_rec:889.000427, tr_eng:-3886.432796, tr_covd:452628.747597, val_AUC:0.506736, val_pAUC:0.496399
100%|██████████| 15/15 [00:02<00:00,  5.06it/s]
2020-11-30 23:07:59,637 - pytorch_modeler.py - INFO - Epoch 31/300:train_loss:1010.907263, tr_rec:1110.073347, tr_eng:-1224.199682, tr_covd:4650.776619, val_AUC:0.500000, val_pAUC:0.500000
100%|██████████| 15/15 [00:02<00:00,  5.63it/s]
2020-11-30 23:08:49,157 - pytorch_modeler.py - INFO - Epoch 32/300:train_loss:732.529280, tr_rec:826.417213, tr_eng:-1104.803148, tr_covd:3318.477993, val_AUC:0.473866, val_pAUC:0.505614
100%|██████████| 15/15 [00:02<00:00,  5.07it/s]
2020-11-30 23:09:39,211 - pytorch_modeler.py - INFO - Epoch 33/300:train_loss:757.355861, tr_rec:934.576091, tr_eng:-2081.835459, tr_covd:6192.664383, val_AUC:0.453936, val_pAUC:0.489740
100%|██████████| 15/15 [00:03<00:00,  4.97it/s]
2020-11

100%|██████████| 15/15 [00:03<00:00,  4.34it/s]
2020-11-30 23:31:14,701 - pytorch_modeler.py - INFO - Epoch 59/300:train_loss:651.082371, tr_rec:878.380129, tr_eng:-2419.911507, tr_covd:2938.679535, val_AUC:0.543517, val_pAUC:0.510701
100%|██████████| 15/15 [00:03<00:00,  4.31it/s]
2020-11-30 23:32:05,759 - pytorch_modeler.py - INFO - Epoch 60/300:train_loss:727.876824, tr_rec:888.972192, tr_eng:-1886.283274, tr_covd:5506.592438, val_AUC:0.503449, val_pAUC:0.504425
100%|██████████| 15/15 [00:03<00:00,  4.26it/s]
2020-11-30 23:32:56,819 - pytorch_modeler.py - INFO - Epoch 61/300:train_loss:788.478131, tr_rec:879.993805, tr_eng:-1076.336901, tr_covd:3223.603008, val_AUC:0.498937, val_pAUC:0.499934
100%|██████████| 15/15 [00:03<00:00,  4.45it/s]
2020-11-30 23:33:47,989 - pytorch_modeler.py - INFO - Epoch 62/300:train_loss:774.324101, tr_rec:866.405977, tr_eng:-1060.812544, tr_covd:2799.875896, val_AUC:0.506035, val_pAUC:0.500102
100%|██████████| 15/15 [00:03<00:00,  4.26it/s]
2020-11-30 2

...

100%|██████████| 15/15 [00:04<00:00,  3.40it/s]
2020-12-01 00:23:26,915 - pytorch_modeler.py - INFO - Epoch 117/300:train_loss:694.527670, tr_rec:799.609602, tr_eng:-1291.155733, tr_covd:4806.729126, val_AUC:0.502509, val_pAUC:0.500134
100%|██████████| 15/15 [00:04<00:00,  3.18it/s]
2020-12-01 00:24:33,524 - pytorch_modeler.py - INFO - Epoch 118/300:train_loss:702.803679, tr_rec:824.136360, tr_eng:-1454.069984, tr_covd:4814.863716, val_AUC:0.464232, val_pAUC:0.496605
100%|██████████| 15/15 [00:04<00:00,  3.21it/s]
2020-12-01 00:25:39,496 - pytorch_modeler.py - INFO - Epoch 119/300:train_loss:578.365638, tr_rec:846.103114, tr_eng:-3195.204344, tr_covd:10356.591736, val_AUC:0.458248, val_pAUC:0.503306
100%|██████████| 15/15 [00:04<00:00,  3.27it/s]
2020-12-01 00:26:45,115 - pytorch_modeler.py - INFO - Epoch 120/300:train_loss:595.535896, tr_rec:836.231304, tr_eng:-2868.555008, tr_covd:9232.019279, val_AUC:0.448593, val_pAUC:0.497036
 24%|██▎       | 12/51 [00:15<00:50,  1.31s/it]
2020-12

error
use: cuda:0


  0%|          | 0/46 [00:01<?, ?it/s]
2020-12-01 00:27:03,656 - 00_train.py - INFO - TARGET MACHINE_TYPE: slider
2020-12-01 00:27:03,656 - 00_train.py - INFO - MAKE DATA_LOADER
2020-12-01 00:27:03,659 - 00_train.py - INFO - TRAINING


error


KeyboardInterrupt: 

history = run('ToyCar')
with open('{}/{}_history.pkl'.format(PKL_DIR, machine_type), 'wb') as file:
    pickle.dump(history , file)